In [76]:
import torch
import pickle
import re
import json
import pandas as pd
from tqdm import tqdm
from transformers import BartForConditionalGeneration, BartTokenizer
from torch.utils.data import DataLoader, Dataset


In [77]:
model_checkpoint = '../../models/fine_tuned_bart_autocorrection'

In [78]:
model = BartForConditionalGeneration.from_pretrained(model_checkpoint)
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

In [79]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [80]:
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [81]:
with open('../../data/test.pickle', 'rb') as f:
    data = pickle.load(f)
data

,wrong_text,correct_text
0,@bessa2204 @ericamfp @mariajdias_ perdoo bessi...,@bessa2204 @ericamfp @mariajdias_ Perdão bessi...
1,@bessa2204 @ericamfp @mariajdias_ perdao bessi...,@bessa2204 @ericamfp @mariajdias_ Perdão bessi...
2,@bessa2204 @ericamfp @mariajdias_ Perdão bessi...,@bessa2204 @ericamfp @mariajdias_ Perdão bessi...
3,Produto chegou bem antes do prazo. Não tive pr...,Produto chegou bem antes do prazo. Não tive pr...
4,Produto chegou bem antes do prazo. Não tive pr...,Produto chegou bem antes do prazo. Não tive pr...
...,...,...
6377085,"Pessoal, chegavam alguns equipamentos novos pr...","Pessoal, chegaram alguns equipamentos novos pr..."
6377086,"Pessoal, chegaram alguns equipamentos novos pr...","Pessoal, chegaram alguns equipamentos novos pr..."
6377087,"Muito insatisfeito, o produto chegou em péssim...","Muito insatisfeito, o produto chegou em péssim..."
6377088,"Muito insatisfeito, o produto chegou em péssim...","Muito insatisfeito, o produto chegou em péssim..."


In [41]:
class AutoCorrectionDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self._data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self._data)

    def __getitem__(self, idx):
        inputs, target = self._data[idx]
        model_inputs = self.tokenizer(inputs, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')
        labels = self.tokenizer(text_target=target, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')
        model_inputs['labels'] = labels['input_ids']

        return {type: data[0] for type, data in model_inputs.items()}

In [85]:
wrong_text = data['wrong_text']
correct_text = data['correct_text']
test_data = list(zip(wrong_text, correct_text))

test_dataset = AutoCorrectionDataset(test_data, tokenizer, max_length=128)

In [43]:
# sanity check
dic = test_dataset[0]
input_ids = dic["input_ids"]
labels = dic["labels"]

print(tokenizer.decode(input_ids, skip_special_tokens=True))
print(tokenizer.decode(labels, skip_special_tokens=True))

Ditta para esculpir uma estátua da madona e do Menino usando um cedro do Líbano.
Ditta para esculpir uma estátua da Madonna e do Menino usando um cedro do Líbano.


In [44]:
test_loader = DataLoader(test_dataset, batch_size=32)

# Vocabulary of mistakes

Save all the bad words that the spellchecker has to fix.

In [86]:
vocab_wrong = {}

In [87]:
for sent_wrong, sent_right in test_data:
    wrong = sent_wrong.split()
    right = sent_right.split()

    for w1, w2 in zip(wrong, right):
        if w1 != w2:
            vocab_wrong[(w1,w2)] = vocab_wrong.get(w1, 0) + 1

In [88]:
vocab_wrong

{('perdoo', 'Perdão'): 1,
 ('perdao', 'Perdão'): 1,
 ('declamar.', 'reclamar.'): 1,
 ('reclamam.', 'reclamar.'): 1,
 ('Legal,', 'legal,'): 1,
 ('ilegal,', 'legal,'): 1,
 ('Amo', 'amo'): 1,
 ('aço', 'amo'): 1,
 ('so', 'do'): 1,
 ('co', 'do'): 1,
 ('foto', 'fato'): 1,
 ('faço', 'fato'): 1,
 ('Trabalho', 'trabalho'): 1,
 ('trabalhos', 'trabalho'): 1,
 ('professar', 'professor'): 1,
 ('professou', 'professor'): 1,
 ('me', 'de'): 1,
 ('se', 'de'): 1,
 ('adaptando', 'adaptado'): 1,
 ('adaptador', 'adaptado'): 1,
 ('Produto,', 'produto,'): 1,
 ('produtor,', 'produto,'): 1,
 ('coupe', 'coube'): 1,
 ('cube', 'coube'): 1,
 ('das', 'da'): 1,
 ('dia', 'da'): 1,
 ('Porta', 'porta'): 1,
 ('posta', 'porta'): 1,
 ('supor', 'super'): 1,
 ('supera', 'super'): 1,
 ('pertenecentes', 'pertenecente'): 1,
 ('https://l.co/f3aQWL39RB', 'https://t.co/f3aQWL39RB'): 1,
 ('https://g.co/f3aQWL39RB', 'https://t.co/f3aQWL39RB'): 1,
 ('ao', 'o'): 1,
 ('ou', 'o'): 1,
 ('irregularis:', 'irregulares:'): 1,
 ('Irregulares

In [89]:
df = {'wrong': [], 'correction': []}

for w1, w2 in vocab_wrong:
    df['wrong'].append(w1)
    df['correction'].append(w2)

In [90]:
df = pd.DataFrame(df)

In [91]:
df

,wrong,correction
0,perdoo,Perdão
1,perdao,Perdão
2,declamar.,reclamar.
3,reclamam.,reclamar.
4,"Legal,","legal,"
...,...,...
451914,t.vo,t.v
451915,"Jae-it,","Jae-in,"
451916,"Jae-is,","Jae-in,"
451917,lb:電気,lo:電気


In [92]:
with open('../../data/test_wrong_vocabulary.pickle', 'wb') as file:
    pickle.dump(df, file)

# Accuracy measurement

In [ ]:
total_samples = 0
total_correct = 0

with torch.no_grad():
    loop = tqdm(test_loader, leave=True)

    for batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        predicted_labels = outputs.logits.argmax(dim=-1)

        # Compare the predicted values with the ground truth labels and count the matches
        correct_mask = (predicted_labels == labels)
        for ix,sample in enumerate(correct_mask):
            if False not in sample:
                total_correct += 1 

        batch_size = batch['input_ids'].shape[0]
        total_samples += batch_size 

        loop.set_postfix({'accuracy': total_correct/total_samples})